In [9]:
from pyspark.sql.types import StructField, StructType, LongType, StringType, IntegerType, DoubleType
ratings = [('A', 'a', 2.0), ('A', 'c', 2.0), ('B', 'a', 1.0), ('B', 'b', 2.0), ('C', 'b', 3.0), ('C', 'c', 5.0)]
mySchema = StructType([ StructField("user_id", StringType(), True) ,StructField("news_id", StringType(), True), StructField("rating", DoubleType(), True)])
ratings = spark.createDataFrame(ratings, schema=mySchema)

rating_by_user_rdd = ratings.rdd.map(lambda x: (x.user_id, (x.news_id, x.rating))).groupByKey().mapValues(list)

In [11]:
rating_cross = rating_by_user_rdd.cartesian(rating_by_user_rdd)

In [12]:
min_common_item_bar = 1

# 1.user_based CF. user_similarity
#common_item < min_common_item_bar则return None
def user_sim_func(xi):
    l1 = xi[0][1]
    l2 = xi[1][1]
    common_item = set([kv[0] for kv in l1]).intersection(set([kv[0] for kv in l2]))
    if len(common_item) >= min_common_item_bar:
        vector_1 = [kv[1] for kv in l1 if kv[0] in common_item]
        vector_2 = [kv[1] for kv in l2 if kv[0] in common_item]
        #保留3位小数
        cos = np.around(cos_sim(vector_1, vector_2),5)
        return (xi[0][0],  (xi[1][0], cos))


def cos_sim(vector_a, vector_b):
    vector_a = np.mat(vector_a)
    vector_b = np.mat(vector_b)
    num = float(vector_a * vector_b.T)
    denom = np.linalg.norm(vector_a) * np.linalg.norm(vector_b)
    cos = num / denom
    return cos


user_sim_rdd = rating_cross.map(user_sim_func).filter(lambda x: x is not None).groupByKey().mapValues(list)

In [23]:
a = sc.parallelize([('B', [('A', 1.0), ('B', 1.0), ('C', 1.0)]),
                    ('C', [('A', 1.0), ('B', 1.0), ('C', 1.0)]),
                    ('A', [('A', 1.0), ('B', 1.0), ('C', 1.0)])])

In [25]:
a.mapValues(lambda s: s).collect()

[('B', [('A', 1.0), ('B', 1.0), ('C', 1.0)]),
 ('C', [('A', 1.0), ('B', 1.0), ('C', 1.0)]),
 ('A', [('A', 1.0), ('B', 1.0), ('C', 1.0)])]

In [26]:
#将相似用户作为key
def f(x): return x
temp = a.flatMapValues(f).map(lambda x:(x[1][0], (x[0], x[1][1])))

In [27]:
temp.collect()

[('A', ('B', 1.0)),
 ('B', ('B', 1.0)),
 ('C', ('B', 1.0)),
 ('A', ('C', 1.0)),
 ('B', ('C', 1.0)),
 ('C', ('C', 1.0)),
 ('A', ('A', 1.0)),
 ('B', ('A', 1.0)),
 ('C', ('A', 1.0))]

In [28]:
rating_by_user_rdd = sc.parallelize([[('A', [('a', 2.0), ('c', 2.0)]),
('B', [('a', 1.0), ('b', 2.0)]),
('C', [('b', 3.0), ('c', 5.0)])]])

In [29]:
temp2 = temp.join(rating_by_user_rdd)

In [32]:
temp2.take(1)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 15.0 failed 1 times, most recent failure: Lost task 3.0 in stage 15.0 (TID 199, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "D:\Users\hao.guo\spark-2.3.2-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 253, in main
  File "D:\Users\hao.guo\spark-2.3.2-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 248, in process
  File "D:\Users\hao.guo\spark-2.3.2-bin-hadoop2.7\python\pyspark\rdd.py", line 2440, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "D:\Users\hao.guo\spark-2.3.2-bin-hadoop2.7\python\pyspark\rdd.py", line 350, in func
    return f(iterator)
  File "D:\Users\hao.guo\spark-2.3.2-bin-hadoop2.7\python\pyspark\rdd.py", line 1943, in combine
    merger.mergeValues(iterator)
  Fi